In [65]:
import altair as alt
import pandas as pd
import numpy as np
import requests
from vega_datasets import data
import json

# 1. Get the URL for the world TopoJSON (countries)
url = data.world_110m.url

# 2. Download and parse it so you can inspect the country IDs
resp     = requests.get(url)
topology = resp.json()

# 3. Drill into the 'countries' object and grab each geometry’s id
#    These IDs are string‐encoded ISO 3166-1 numeric codes
geoms       = topology['objects']['countries']['geometries']
country_ids = [geom['id'] for geom in geoms]  # these are strings already

# 4. Read in your data
fh = pd.read_excel(r"C:\Users\ayaan\OneDrive - London School of Economics\Activities\The Metrician\themetrician\viz\FreedomHouse Data.xlsx", sheet_name="FIW13-25")
iso = pd.read_csv(r"C:\Users\ayaan\OneDrive - London School of Economics\Activities\The Metrician\themetrician\viz\ISO-3166.csv")

# 5. Filter to 2025 and merge to get the numeric code
df25 = fh[fh['Edition'] == 2025][['Country/Territory', 'Total']]
df   = pd.merge(
    df25,
    iso[['name', 'country-code']],
    left_on="Country/Territory",
    right_on="name",
    how="inner"
)

# 6. Build the lookup DataFrame – convert country-code to string!
# 6. Build the lookup DataFrame – note we call the column "Country"
df1 = pd.DataFrame({
    'id':                     df["country-code"].astype(str),
    'Freedom House Score':    df["Total"],
    'Country':                df["Country/Territory"]
})

# 7. Build your Vega-Lite chart spec via Altair
chart = (
    alt.Chart(alt.topo_feature(url, feature='countries'))
      .mark_geoshape(stroke='white', strokeWidth=0.3)
      .encode(
          color='Freedom House Score:Q',
          tooltip=[
            'Country:N',
            'Freedom House Score:Q'
          ]
      )
      .transform_lookup(
          lookup='id',
          from_=alt.LookupData(
            df1,               # your lookup DataFrame
            key='id',          # the field in df1
            fields=['Freedom House Score','Country']
          )
      )
      .project('equalEarth')
      .properties(width=800, height=450)
)

chart

# 9. Export to JSON for your repo
chart.save(r"C:\Users\ayaan\OneDrive - London School of Economics\Activities\The Metrician\themetrician\freedomhouse2025_map.json")